In [1]:
from typing import Optional
import re
import pandas as pd

In [2]:
pd.options.display.max_columns = 30

In [3]:
def read_and_rename(path: str) -> Optional[pd.DataFrame]:
    try:
        df = pd.read_csv(path)
    except(FileNotFoundError):
        print("File does not exists!")
        return None
    
    #basic columns renaming
    rename_mapper = {k:re.sub("[^A-Z|_]", "", k.lower().replace(" ", "_") ,0,re.IGNORECASE) for k in df.columns}
    return df.rename(columns=rename_mapper)
    
    

In [4]:
df_brighton = read_and_rename("BrightonPerformanceData.csv")
geo_columns = ['latitude', 'longitude', 'zipcode', 'city']
constants = ['scraped_during_month', 'country_code', 'currency_native']
cols_to_drop = geo_columns + constants
df_brighton = df_brighton.drop(cols_to_drop, axis=1)
# df_joshua = pd.read_csv("JoshuaPerformanceData.csv")
# df_malibu = pd.read_csv("MalibuPerformanceData.csv")

### basic profiling on dataset

```python
# from pydantic_settings import BaseSettings
from ydata_profiling import ProfileReport

#EDA using pandas-profiling
profile = ProfileReport(df_brighton, explorative=True)

#Saving results to a HTML file
profile.to_file("output_brighton.html") 
```

In [5]:
df_brighton['num_reporting_months'] = df_brighton[["airbnb_property_id", "reporting_month"]].groupby("airbnb_property_id").transform("count")


distinct_last_seen = df_brighton.groupby('airbnb_property_id')['last_seen'].nunique()
df_brighton['distinct_last_seen'] = df_brighton['airbnb_property_id'].map(distinct_last_seen)

distinct_reporting_month = df_brighton.groupby('airbnb_property_id')['reporting_month'].nunique()
df_brighton['distinct_reporting_month'] = df_brighton['airbnb_property_id'].map(distinct_reporting_month)

In [6]:
# Convert 'month' column to datetime format
df_brighton['reporting_month'] = pd.to_datetime(df_brighton['reporting_month'])

# Sort the DataFrame by 'id' and 'month'
df_brighton.sort_values(by=['airbnb_property_id', 'reporting_month'], inplace=True)

# Calculate the difference in days between consecutive rows for each 'airbnb_property_id'
df_brighton['diff'] = df_brighton.groupby('airbnb_property_id')['reporting_month'].diff().dt.days
#filling for the first row
df_brighton['diff'] = df_brighton['diff'].fillna(0.)

# Check if the difference is equal to 1 for continuous rows
df_brighton['continuous'] = df_brighton['diff'] <= 31

df_brighton[['airbnb_property_id', 'reporting_month', 'diff', 'continuous', 'num_reporting_months']][df_brighton.continuous == True]


,airbnb_property_id,reporting_month,diff,continuous,num_reporting_months
23507,74819,2022-11-01,0.0,True,12
22179,74819,2022-12-01,30.0,True,12
20740,74819,2023-01-01,31.0,True,12
19052,74819,2023-02-01,31.0,True,12
17150,74819,2023-03-01,28.0,True,12
...,...,...,...,...,...
4991,985326724713973239,2023-09-01,0.0,True,2
2316,985326724713973239,2023-10-01,30.0,True,2
4992,985389662447322939,2023-09-01,0.0,True,2
2317,985389662447322939,2023-10-01,30.0,True,2


In [7]:
#how many properties has continuous bookings?
df_brighton[['airbnb_property_id', 'continuous']].drop_duplicates().continuous.value_counts()#.groupby("continuous").transform("count")

continuous
True     3928
False     757
Name: count, dtype: int64

In [8]:
df_brighton['computed_bedrooms'] = df_brighton['bedrooms'].replace("Studio", "1").astype("int64")
df_brighton['rank'] = df_brighton['adr_native']/(df_brighton['computed_bedrooms']+ df_brighton['bathrooms'])

In [9]:
df_brighton['mean_rank'] = df_brighton.groupby("airbnb_property_id")['rank'].transform('mean')
df_brighton['mean_adr'] = df_brighton.groupby("airbnb_property_id")['adr_native'].transform('mean')
df_brighton['mean_or'] = df_brighton.groupby("airbnb_property_id")['occupancy_rate'].transform('mean')

In [10]:
pd.options.display.max_rows = 100

In [11]:
distinct_occurences = df_brighton.groupby('reporting_month')['airbnb_property_id'].nunique()
df_brighton['distinct_occurences'] = df_brighton['reporting_month'].map(distinct_occurences)


In [20]:
df_brighton['last_month'] = df_brighton.groupby("airbnb_property_id")['reporting_month'].transform("max")
df_brighton['dead'] = df_brighton['last_month'] + pd.tseries.offsets.DateOffset(months=2) < '2023-10-01'

,property_type,listing_type,bedrooms,bathrooms,airbnb_property_id,airbnb_host_id,last_seen,cleaning_fee,reporting_month,blocked_days,available_days,occupancy_rate,reservation_days,adr_usd,adr_native,number_of_reservation,revenue_usd,revenue_native,next_month_exists
23507,Entire home,entire_home,2,2,74819,394634,2024-01-10,46.0,2022-11,1,30,52.0,13,229,181,8,3345,2643,True
22179,Entire home,entire_home,2,2,74819,394634,2024-01-10,46.0,2022-12,0,31,61.9,16,279,220,9,4878,3854,True
20740,Entire home,entire_home,2,2,74819,394634,2024-01-10,46.0,2023-01,0,31,100.0,31,255,201,5,8135,6427,True
19052,Entire home,entire_home,2,2,74819,394634,2024-01-10,46.0,2023-02,3,28,100.0,28,225,178,5,6530,5159,True
17150,Entire home,entire_home,2,2,74819,394634,2024-01-10,46.0,2023-03,4,27,100.0,24,230,182,8,5888,4652,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,Entire rental unit,entire_home,2,2,985326724713973239,107736029,2024-01-03,NaN,2023-09,28,3,80.0,2,146,115,0,292,231,True
2316,Entire rental unit,entire_home,2,2,985326724713973239,107736029,2024-01-03,NaN,2023-10,0,31,54.2,14,147,116,3,2058,1626,False
4992,Entire rental unit,entire_home,1,1,985389662447322939,538216244,2023-11-29,NaN,2023-09,28,3,80.0,2,238,188,0,476,376,True
2317,Entire rental unit,entire_home,1,1,985389662447322939,538216244,2023-11-29,NaN,2023-10,0,31,31.0,8,160,126,3,1280,1011,False


,property_type,listing_type,bedrooms,bathrooms,airbnb_property_id,airbnb_host_id,last_seen,cleaning_fee,reporting_month,blocked_days,available_days,occupancy_rate,reservation_days,adr_usd,adr_native,number_of_reservation,revenue_usd,revenue_native,next_reporting_month,next_month_exists
23507,Entire home,entire_home,2,2,74819,394634,2024-01-10,46.0,2022-11,1,30,52.0,13,229,181,8,3345,2643,2022-12,True
22179,Entire home,entire_home,2,2,74819,394634,2024-01-10,46.0,2022-12,0,31,61.9,16,279,220,9,4878,3854,2023-01,True
20740,Entire home,entire_home,2,2,74819,394634,2024-01-10,46.0,2023-01,0,31,100.0,31,255,201,5,8135,6427,2023-02,True
19052,Entire home,entire_home,2,2,74819,394634,2024-01-10,46.0,2023-02,3,28,100.0,28,225,178,5,6530,5159,2023-03,True
17150,Entire home,entire_home,2,2,74819,394634,2024-01-10,46.0,2023-03,4,27,100.0,24,230,182,8,5888,4652,2023-04,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4991,Entire rental unit,entire_home,2,2,985326724713973239,107736029,2024-01-03,NaN,2023-09,28,3,80.0,2,146,115,0,292,231,2023-10,True
2316,Entire rental unit,entire_home,2,2,985326724713973239,107736029,2024-01-03,NaN,2023-10,0,31,54.2,14,147,116,3,2058,1626,NaN,False
4992,Entire rental unit,entire_home,1,1,985389662447322939,538216244,2023-11-29,NaN,2023-09,28,3,80.0,2,238,188,0,476,376,2023-10,True
2317,Entire rental unit,entire_home,1,1,985389662447322939,538216244,2023-11-29,NaN,2023-10,0,31,31.0,8,160,126,3,1280,1011,NaN,False


In [37]:
# df_brighton[(df_brighton.reporting_month != '2023-10-01') & (df_brighton.next_month_exists == False) ]\
df_brighton.sort_values(by=['airbnb_property_id', 'reporting_month'])[['airbnb_property_id', 'continuous', 'reporting_month', 'next_reporting_month', 'next_month_exists']].head(50)

,airbnb_property_id,continuous,reporting_month,next_reporting_month,next_month_exists
23507,74819,True,2022-11-01,2022-12-01,True
22179,74819,True,2022-12-01,2023-01-01,True
20740,74819,True,2023-01-01,2023-02-01,True
19052,74819,True,2023-02-01,2023-03-01,True
17150,74819,True,2023-03-01,2023-04-01,True
15152,74819,True,2023-04-01,2023-05-01,True
13135,74819,True,2023-05-01,2023-06-01,True
10745,74819,True,2023-06-01,2023-07-01,True
8209,74819,True,2023-07-01,2023-08-01,True
4993,74819,True,2023-08-01,2023-09-01,True


In [21]:
df_brighton[df_brighton.dead == True][['airbnb_property_id', 'last_month']].drop_duplicates().last_month.value_counts()

last_month
2023-07-01    127
2023-06-01     59
2023-05-01     55
2023-04-01     55
2023-01-01     53
2023-03-01     46
2022-12-01     36
2022-11-01     32
2023-02-01     29
Name: count, dtype: int64

In [28]:
df_brighton.sort_values(by=['airbnb_property_id','reporting_month'])[['airbnb_property_id','reporting_month', 'dead']].head(100)

,airbnb_property_id,reporting_month,dead
23507,74819,2022-11-01,False
22179,74819,2022-12-01,False
20740,74819,2023-01-01,False
19052,74819,2023-02-01,False
17150,74819,2023-03-01,False
15152,74819,2023-04-01,False
13135,74819,2023-05-01,False
10745,74819,2023-06-01,False
8209,74819,2023-07-01,False
4993,74819,2023-08-01,False


In [189]:
df_brighton[['reporting_month', 'distinct_occurences']].drop_duplicates().sort_values(by='reporting_month').head(100)

,reporting_month,distinct_occurences
23507,2022-11-01,1342
22179,2022-12-01,1328
20740,2023-01-01,1439
19052,2023-02-01,1688
17150,2023-03-01,1902
15152,2023-04-01,1998
13135,2023-05-01,2017
10745,2023-06-01,2390
8209,2023-07-01,2536
4993,2023-08-01,3216


In [184]:
df_brighton.groupby("reporting_month")['airbnb_property_id'].transform('count')

23507    1342
22179    1328
20740    1439
19052    1688
17150    1902
         ... 
4991     2674
2316     2319
4992     2674
2317     2319
2318     2319
Name: airbnb_property_id, Length: 24849, dtype: int64

In [181]:
df_brighton[['airbnb_property_id', 'property_type','bedrooms','bathrooms','mean_rank','mean_adr', 'mean_or', 'distinct_reporting_month', 'continuous','last_seen']]\
    .drop_duplicates().sort_values(by='mean_rank', ascending=False).head(100)

,airbnb_property_id,property_type,bedrooms,bathrooms,mean_rank,mean_adr,mean_or,distinct_reporting_month,continuous,last_seen
23948,23936388,Entire guest suite,Studio,1,279.785714,559.571429,47.657143,7,True,2023-03-16
11300,23936388,Entire guest suite,Studio,1,279.785714,559.571429,47.657143,7,False,2023-03-16
15272,5087973,Entire townhouse,1,1,265.785714,531.571429,62.400000,7,False,2024-01-10
23613,5087973,Entire townhouse,1,1,265.785714,531.571429,62.400000,7,True,2024-01-10
10327,723865952947832147,Entire condo,1,1,257.375000,514.750000,53.300000,4,True,2024-01-03
24727,51833929,Entire home,3,2,218.472727,1092.363636,51.072727,11,True,2024-01-03
3705,51833929,Entire home,3,2,218.472727,1092.363636,51.072727,11,False,2024-01-03
2268,957952560777806390,Entire rental unit,2,1,211.333333,634.000000,78.300000,1,True,2023-11-16
24724,51833680,Entire home,1,1,210.750000,421.500000,33.000000,2,True,2024-01-03
3704,51833829,Entire rental unit,1,1,199.318182,398.636364,36.909091,11,False,2024-01-03
